# Dragon Lake Parking (DLP) Dataset

This notebook walks through usage of the Parking Dataset API.

### Setup

The `load()` method loads all JSON files for a scene. In this example, we are loading:

- `data/DJI_0001_frames.json`
- `data/DJI_0001_agents.json`
- `data/DJI_0001_instances.json`
- `data/DJI_0001_obstacles.json`
- `data/DJI_0001_scene.json`

In [17]:
from my_dlp.dataset import Dataset

ds = Dataset()
ds.load('../data/DJI_0012')

### Scenes

We can use `list_scenes()` to obtain a list of tokens corresponding to scenes that have been loaded.

In [18]:
all_scenes = ds.list_scenes()
all_scenes

['scene_0']

Use `get()` to look up a data object by its token.

In [19]:
scene_token = all_scenes[0]
scene = ds.get('scene', scene_token)
print("Keys:", scene.keys())
print("Scene_token:", scene['scene_token'])
print("filename:", scene['filename'])
print("timestamp:", scene['timestamp'])
print("first_frame:", scene['first_frame'])
print("last_frame:", scene['last_frame'])
print("agents (preview):", scene['agents'][:2])
print("obstacles (preview):", scene['obstacles'][:2])

Keys: dict_keys(['scene_token', 'filename', 'timestamp', 'first_frame', 'last_frame', 'agents', 'obstacles'])
Scene_token: scene_0
filename: DJI_0012
timestamp: 2020-08-08 10:08:03
first_frame: frame_0
last_frame: frame_100
agents (preview): ['agent_0', 'agent_1']
obstacles (preview): ['02778fb4cafec30d01ba4e59a8eb53f19c3b9c86', '8d08ec553539276f33cba6aa12eed8fc07371672']


### Frames

Let's look at the first frame in the scene:

In [20]:
frame_token = scene['first_frame']
frame = ds.get('frame', frame_token)
print("Keys:", frame.keys())
print("frame_token:", frame['frame_token'])
print("scene_token:", frame['scene_token'])
print("timestamp:", frame['timestamp'])
print("instances (preview):", frame['instances'][:2])

Keys: dict_keys(['frame_token', 'scene_token', 'timestamp', 'next', 'prev', 'instances'])
frame_token: frame_0
scene_token: scene_0
timestamp: 0.0
instances (preview): ['frame_0_instance_agent_0', 'frame_0_instance_agent_1']


We can use the `get_frame_at_time()` method to get the frame at a certian time. The specified time to query can be any float value (in secs) within the total length of the video, and the method will return the closet frame towards that time. The example below returns the frame at about 15.00s of the video. 

In [21]:
frame = ds.get_frame_at_time(scene_token, 2)
print("Keys:", frame.keys())
print("frame_token:", frame['frame_token'])
print("scene_token:", frame['scene_token'])
print("timestamp:", frame['timestamp'])
print("instances (preview):", frame['instances'][:2])

Keys: dict_keys(['frame_token', 'scene_token', 'timestamp', 'next', 'prev', 'instances'])
frame_token: frame_50
scene_token: scene_0
timestamp: 2.0
instances (preview): ['frame_50_instance_agent_0', 'frame_50_instance_agent_1']


We can use the `get_future_frames()` or `get_past_frames()` methods look at a specified number of preceding/subsequent frames. The example below returns a list containing the given frame and the next 5 frames.

In [22]:
future_frames = ds.get_future_frames(frame_token, timesteps=5)
print("Keys:", future_frames[0].keys())
print(len(future_frames), "future frames")


Keys: dict_keys(['frame_token', 'scene_token', 'timestamp', 'next', 'prev', 'instances'])
6 future frames


### Agents

Now let's look at an agent:

In [23]:
agent_token = scene['agents'][0]
agent = ds.get('agent', agent_token)
agent

{'agent_token': 'agent_0',
 'scene_token': 'scene_0',
 'type': 'Car',
 'size': [4.7048, 1.8778],
 'first_instance': 'frame_0_instance_agent_0',
 'last_instance': 'frame_100_instance_agent_0'}

Use `get_agent_instances()` to see all instances of an agent in the scene.

In [24]:
agent_instances = ds.get_agent_instances(agent_token)

print("Keys:", agent_instances[0].keys())
print("coords:", agent_instances[0]['coords'])
print("heading:", agent_instances[0]['heading'])
print("speed:", agent_instances[0]['speed'])
print("acceleration:", agent_instances[0]['acceleration'])
print("mode:", agent_instances[0]['mode'])
print("prev:", agent_instances[0]['prev'])
print("next:", agent_instances[0]['next'])

Keys: dict_keys(['frame_token', 'agent_token', 'instance_token', 'coords', 'heading', 'speed', 'acceleration', 'mode', 'prev', 'next'])
coords: [77.70538462, 64.95]
heading: 0.0
speed: 0.0
acceleration: [0.0, 0.0]
mode: unclear
prev: None
next: frame_1_instance_agent_0


### Instances

This is the last instance of our agent in the scene:

In [25]:
instance_token = agent['last_instance']
instance = ds.get('instance', instance_token)
instance

{'frame_token': 'frame_0',
 'agent_token': 'agent_0',
 'instance_token': 'frame_0_instance_agent_0',
 'coords': [77.70538462, 64.95],
 'heading': 0.0,
 'speed': 0.0,
 'acceleration': [0.0, 0.0],
 'mode': 'unclear',
 'prev': None,
 'next': 'frame_1_instance_agent_0'}

We can search for the closest instance at a certain location with `get_inst_at_location()` method:

In [26]:
closest_instance = ds.get_inst_at_location(frame_token, [90, 65])
closest_instance

{'frame_token': 'frame_0',
 'agent_token': 'agent_4',
 'instance_token': 'frame_0_instance_agent_4',
 'coords': [100.5, 64.95],
 'heading': 3.14,
 'speed': 0.0,
 'acceleration': [0.0, 0.0],
 'mode': 'unclear',
 'prev': None,
 'next': 'frame_1_instance_agent_4'}

We can use the `get_agent_future()` or `get_agent_past()` methods to look at a specified number of future/past instances of this agent. The example below returns a list containing the previous 5 instances along with the given instance.

In [27]:
past_instances = ds.get_agent_past(instance_token, timesteps=5)

print("Keys:", past_instances[0].keys())
print("coords:", past_instances[0]['coords'])
print("heading:", past_instances[0]['heading'])
print("speed:", past_instances[0]['speed'])
print("acceleration:", past_instances[0]['acceleration'])
print("mode:", past_instances[0]['mode'])
print("prev:", past_instances[0]['prev'])
print("next:", past_instances[0]['next'])

Keys: dict_keys(['frame_token', 'agent_token', 'instance_token', 'coords', 'heading', 'speed', 'acceleration', 'mode', 'prev', 'next'])
coords: [77.70538462, 64.95]
heading: 0.0
speed: 0.0
acceleration: [0.0, 0.0]
mode: unclear
prev: None
next: frame_1_instance_agent_0


By default, the speed value in the instance is the absolute value. To obtain the "signed" speed so that it is negative when vehicle is backing up, use `signed_speed()` function

In [28]:
signed_speed = ds.signed_speed(inst_token=instance_token)
print('signed_speed:', signed_speed)

signed_speed: 0.0


Besides a list of future instances, it is also possible to get an Tx4 numpy array as the vehicle's future trajectory

In [29]:
future_traj = ds.get_future_traj(inst_token=instance_token)
print(future_traj)

[[77.74538462 64.95        0.          1.        ]
 [77.78538462 64.95        0.          1.        ]
 [77.82538462 64.95        0.          1.        ]
 [77.86538462 64.95        0.          1.        ]
 [77.90538462 64.95        0.          1.        ]
 [77.94538462 64.95        0.          1.        ]
 [77.98538462 64.95        0.          1.        ]
 [78.02538462 64.95        0.          1.        ]
 [78.06538462 64.95        0.          1.        ]
 [78.10538462 64.95        0.          1.        ]
 [78.14538462 64.95        0.          1.        ]
 [78.18538462 64.95        0.          1.        ]
 [78.22538462 64.95        0.          1.        ]
 [78.26538462 64.95        0.          1.        ]
 [78.30538462 64.95        0.          1.        ]
 [78.34538462 64.95        0.          1.        ]
 [78.38538462 64.95        0.          1.        ]
 [78.42538462 64.95        0.          1.        ]
 [78.46538462 64.95        0.          1.        ]
 [78.50538462 64.95        0.  

An instance will have one of the stree modes:
* 'parked': if the current instance is just static inside a parking spot
* 'outgoing`: if the currrent instance is on the way towards the exit
* 'incoming`: if the current instance is searching for a parking spot
* 'unclear`: we cannot tell the vehicle's intent given the data recording

The `get_inst_mode()` function will not only return the mode of the specified instance, but also write the mode into the instance object for later use

In [30]:
mode = ds.get_inst_mode(inst_token=instance_token)
print('returned mode:', mode)
instance

returned mode: unclear


{'frame_token': 'frame_0',
 'agent_token': 'agent_0',
 'instance_token': 'frame_0_instance_agent_0',
 'coords': [77.70538462, 64.95],
 'heading': 0.0,
 'speed': 0.0,
 'acceleration': [0.0, 0.0],
 'mode': 'unclear',
 'prev': None,
 'next': 'frame_1_instance_agent_0'}

### Obstacles

The obstacles are the vehicles which stays static all the time in its scene

In [31]:
print("obstacles (preview):", scene['obstacles'][:3])

obstacles (preview): ['02778fb4cafec30d01ba4e59a8eb53f19c3b9c86', '8d08ec553539276f33cba6aa12eed8fc07371672', '49d7b7d2ac181f8b114f21d8d31dab3e314ccc45']


We can use the `get()` method to obtain each obstacle. Each obstacle contains the token of itself, the token of its scene, the type of this obstacle, the size of its bounding box, the center coordinates, and the heading angle

In [32]:
obstacle = ds.get('obstacle', scene['obstacles'][0])
print(obstacle)

{'obstacle_token': '02778fb4cafec30d01ba4e59a8eb53f19c3b9c86', 'scene_token': '66045def24565e370dff814aa01e8ffed07318b9', 'type': 'Bus', 'size': [10.5271, 3.0816], 'coords': [132.93999999994412, 55.31999999983236], 'heading': -1.5545926535897931}
